In [10]:
## 국회의원 현황정보 수집/분석/시각화/저장
- 국회 의원현황
- 이름과 ID 추출
- 상세페이지 정보 추출 (json으로 저장)
- 상세정보들을 DataFrame으로 변환
- 시각화(막대그래프,히스토그램, 파이챠트, 히트맵)
- 테이블로 저장

SyntaxError: invalid syntax (98596494.py, line 2)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

id_list_300 = []
for index in range(1,31): # 1 ~ 30
    url = f'https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex={index}'
    print(url)
    res = requests.get(url, headers=headers)
    #print(res.status_code)
    if res.ok:
        #print(res.json()['resultList'])
        id_list_10 = []
        for member in res.json()['resultList']:
            #print(member['openNaId'])
            id_list_10.append(member['openNaId']) 
        
        #print(id_list_10)
        id_list_300.extend(id_list_10)
    else:
        print(res.status_code)
print(len(id_list_300))
print(id_list_300)

https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=1
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=2
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=3
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=4
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=5
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=6
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=7
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=8
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=9
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=10
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=11
https://www.assembly.go.kr/portal/cnts/cntsNaas/findNaasThx01.json?pageIndex=12
https://www.assembly.go.kr/portal/cnts/cntsNaas/f

#### Json 일부 데이터와 웹페이지의 상세정보 데이터 가져오기

In [11]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('===> 스크래핑 시작')
#297명의 정보를 저장하는 List
member_detail_list = []

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

for idx,member in enumerate(member_list_300,1):
    mem_sub_link = member['mLinkUrl']
    detail_url = f'https://www.assembly.go.kr{mem_sub_link}'
    res = requests.get(detail_url, headers=headers)
    print(res.ok, idx, detail_url)
    
    if res.ok:
        # 응답값의 텍스트를 추출
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 정보를 저장하는 dict 선언
        member_detail_dict = {}
        
        li_tags = soup.select('ul.list li:nth-of-type(n+4)')
        #print(li_tags)
        # for li in li_tags:
        #     print(type(li.find('dt')))
        #     print(li.find('dt').text)
        
        # https://www.assembly.go.kr/members/22nd/KANGKYUNGSOOK 예시
        dt_list = [li_tag.find('dt').text for li_tag in li_tags]
        #print(dt_list)

        dd_list = []        
        for li_tag in li_tags:
            dd_tag = li_tag.find('dd')
            pattern = re.compile(f'[\n\r\t]') #문자열에서 줄바꿈, 캐리지 리턴, 탭 문자 중 하나라도 포함되어 있으면 매칭
            dd_text = pattern.sub('',dd_tag.text.strip()).replace(" ","") #dd_tag.text에서 특정 공백 문자들을 제거하고, 나중엔 띄어쓰기까지 없애는 작업
            dd_list.append(dd_text)
        #print(dd_list)
        
        #dt_list와 dd_list를 쌍(pair) 으로 묶어줌, key vlaue 느낌
        member_detail_dict = dict(zip(dt_list, dd_list))
        
        member_detail_dict['이름'] = member['hgNm']        
        member_detail_dict['정당'] = member['polyNm']
        member_detail_dict['성별'] = member['sexGbnNm']
        member_detail_dict['선거구'] = member['origNm']
        member_detail_dict['지역비례'] = member['electGbnNm']

        sosok = member['cmitNm']
        if sosok is not None:
            idx = sosok.find(',')
            if idx > -1:            
                #쉼표가 존재하면, 첫 번째 위원회만 저장함. 예: '정무위원회,운영위원회' → '정무위원회'
                member_detail_dict['소속위원회'] = sosok[:idx]
            else:
                member_detail_dict['소속위원회'] = sosok
        else:        
            member_detail_dict['소속위원회'] = ''    

        print(member_detail_dict['소속위원회'])                        
        
        member_detail_dict['당선횟수'] = member['reeleGbnNm']
        
        img_sub_url = member['mThumbImgUrl']
        '''
        detail_url = 'https://www.assembly.go.kr/members/22nd/KANGKYUNGSOOK',
        img_sub_url = '/static/portal/img/openassm/new/thumb/68f83caf0c9e4546b1694ead863f45ce.jpg'

        => https://www.assembly.go.kr/static/portal/img/openassm/new/thumb/68f83caf0c9e4546b1694ead863f45ce.jpg 

        절대 경로로 변경해준다.
        '''
        img_url = urljoin(detail_url,img_sub_url)
        #print(img_url)
        member_detail_dict['이미지'] = img_url
        
        #print(member_detail_dict)        
                        
        #1명의 정보가 저장된 dict를 list에 추가하기
        member_detail_list.append(member_detail_dict)
    else:
        print(res.status_code)
        
print(len(member_detail_list))
#print(member_detail_list[298:])
print('===> 스크래핑 끝')

===> 스크래핑 시작
True 1 https://www.assembly.go.kr/members/22nd/KANGKYUNGSOOK
교육위원회
True 2 https://www.assembly.go.kr/members/22nd/KANGDAESIK
국방위원회


KeyboardInterrupt: 

In [12]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('===> 스크래핑 시작')
#297명의 정보를 저장하는 List
member_detail_list = []

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

for idx,mem_id in enumerate(id_list_300,1):
    detail_url = f'https://www.assembly.go.kr/members/22st/{mem_id}'
    res = requests.get(detail_url, headers=headers)
    #print(res.ok, idx, detail_url)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 정보를 저장하는 dict 선언
        member_detail_dict = {}
        
        dt_list = [dt_tag.text for dt_tag in soup.select('ul.list li dt')]
        #print(dt_list)
        
        dd_list = []        
        for dd_tag in soup.select('ul.list li dd'):
            pattern = re.compile(f'[\n\r\t]')
            dd_text = pattern.sub('',dd_tag.text.strip()).replace(" ","")
            dd_list.append(dd_text)
        #print(dd_list)
        
        member_detail_dict = dict(zip(dt_list, dd_list))
        #print(member_detail_dict)
        
        # <strong>홍길동 (洪吉童)</strong>
        name = soup.select_one('div.intro p.tit strong').text
        #print(name.split(' ')[0])
        member_detail_dict['이름'] = name.split(' ')[0] #['박주민','(朴柱民)']
        
        span_tag_age = soup.select_one('div.intro p.tit span.sm span')
        member_detail_dict['생년월일'] = span_tag_age.text
        
        spanimg_tag = soup.select('span.img-set span.img')
        if spanimg_tag:
            style_url = spanimg_tag[0]['style']
            start_idx = style_url.find("(")+1
            end_idx = style_url.find(")")
            img_sub_url = style_url[start_idx:end_idx].replace("'","")
            img_url = urljoin(detail_url,img_sub_url)
            #print(img_url)
            member_detail_dict['이미지'] = img_url
        
        dd_tag_party = soup.select_one('div.tit dl dd')
        member_detail_dict['정당'] =  dd_tag_party.text
                
        #1명의 정보가 저장된 dict를 list에 추가하기
        member_detail_list.append(member_detail_dict)
    else:
        print(res.status_code)
        
print(len(member_detail_list))        
print('===> 스크래핑 끝')

===> 스크래핑 시작


AttributeError: 'NoneType' object has no attribute 'text'